In [22]:
source(".Rprofile")
library(scRepertoire)
library(magrittr)
library(bench)
library(data.table)
source("deps.R")

In [23]:
datasets <- get_datasets()
summary(datasets)

        Length Class      Mode
1000    36     data.frame list
3000    36     data.frame list
10000   36     data.frame list
30000   36     data.frame list
100000  36     data.frame list
300000  36     data.frame list
600000  36     data.frame list
1000000 36     data.frame list

In [24]:
process_dataset <- function(dataset) {
    scRepertoire::combineTCR(scRepertoire::loadContigs(dataset, format = "ParseBio"))
}

In [ ]:
benchmark_results <- dplyr::bind_rows(lapply(seq_along(datasets), function(i) {
    gc(verbose = FALSE)
    dataset <- datasets[[i]]
    benchmark_record <- get_bench_record(suppressWarnings(bench::mark(process_dataset(dataset))))
    benchmark_record %>%
        dplyr::mutate(dataset_size = names(datasets[i])) %>%
        dplyr::relocate(dataset_size)
}))
benchmark_results

Warning message:
“Some expressions had a GC in every iteration; so filtering is disabled.”
Warning message:
“Some expressions had a GC in every iteration; so filtering is disabled.”
Warning message:
“Some expressions had a GC in every iteration; so filtering is disabled.”
Warning message:
“Some expressions had a GC in every iteration; so filtering is disabled.”


dataset_size,min,median,num_iter,mem_alloc,num_gc,gc0_mean,gc0_median,gc0_min,gc0_max,gc1_mean,gc1_median,gc1_min,gc1_max,gc2_mean,gc2_median,gc2_min,gc2_max
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1000,42.3ms,47.5ms,11,8.93MB,0,0,0,0,0,0,0,0,0,0,0,0,0
3000,113ms,118ms,5,26.3MB,0,0,0,0,0,0,0,0,0,0,0,0,0
10000,343ms,371ms,2,87.4MB,0,0,0,0,0,0,0,0,0,0,0,0,0
30000,1.19s,1.19s,1,266MB,0,0,0,0,0,0,0,0,0,0,0,0,0
100000,5.15s,5.15s,1,864MB,1,1,1,1,1,0,0,0,0,0,0,0,0
300000,18.7s,18.7s,1,2.56GB,4,3,3,3,3,1,1,1,1,0,0,0,0
600000,41.4s,41.4s,1,5.13GB,8,4,4,4,4,3,3,3,3,1,1,1,1
1000000,1.27m,1.27m,1,8.64GB,10,5,5,5,5,3,3,3,3,2,2,2,2


In [79]:
fwrite(benchmark_results, "results/scRepertoire2.csv")

In [86]:
library(profvis)
library(htmlwidgets)
saveWidget(profvis(scRepertoire::loadContigs(datasets[["300000"]], format = "ParseBio")), "loadContigs.html", selfcontained = TRUE)

x <- scRepertoire::loadContigs(datasets[["300000"]], format="ParseBio")

saveWidget(profvis(scRepertoire::combineTCR(x)), "combineTCR.html", selfcontained = TRUE)